In [3]:
import numpy as np
import pandas as pd
import os
from datetime import datetime
import statistics
import pickle
import csv
from nilearn import plotting
from scipy import stats
import nibabel as nib

In [5]:
def make_4d(dir_list, output_name):
    #Loop through each of the 3D scan paths in dir_list and make a 4D array out of them
    #First, get the shape of the 3D array
    img = nib.load(dir_list[0])
    data = img.get_fdata()
    shape = data.shape
    #Then, make a 4D array of zeros
    fourd_data = np.zeros((shape[0], shape[1], shape[2], len(dir_list)))
    #Then, loop through each 3D array and add it to the 4D array
    for i in range(len(dir_list)):
        print(dir_list[i])
        img = nib.load(dir_list[i])
        data = img.get_fdata()
        fourd_data[:,:,:,i] = data
    #Then save the 4D array as a nifti file
    affine = img.affine
    header = img.header
    new_img = nib.Nifti1Image(fourd_data, affine, header)
    nib.save(new_img, output_name)
    print("Saved 4D array as ", output_name)

In [6]:
#Converts CSVs to the mat format for FSL randomise.
def csv_to_mat(csv_path, out_path):
    
    with open(csv_path) as f:
        data = list(csv.reader(f))

    del data[0] #removes column titles
    num_row = str(len(data))
    num_col = str(len(data[0]))
    
    #write headers
    os.system("echo /NumWaves " +  num_col + " > " + out_path)
    os.system("echo /NumPoints " +  num_row + " >> " + out_path)
    os.system("echo /Matrix >> " + out_path)
    
    for line in data:
        line_txt = ' '.join(line)
        os.system("echo " +  line_txt + " >> " + out_path)
    print("Generated " + out_path)

# Patients vs. controls at baseline

In [9]:
#Read in the DF sorted by PATNO
sorted_table = pd.read_pickle('table.pickle')

# Make GLM T-test design matrix
n_subjects = sorted_table.PATNO.unique().size
mini_df = sorted_table[['PATNO', 'EVENT_ID', 'AGE_AT_VISIT', 'SEX', 'COHORT']].copy()
mini_df = mini_df.loc[mini_df.EVENT_ID == "BL"]# Only keep baseline scans
cohort_temp = mini_df['COHORT'] - 1 #Make cohorts 1(HC) and 0(PD) (as opposed to 2 and 1).
mini_df['COHORT'] = cohort_temp
mini_df = mini_df.drop('EVENT_ID', axis=1)
mini_df = mini_df.loc[mini_df["PATNO"] != 3857] #remove corrupted scans
mini_df = mini_df.loc[mini_df["PATNO"] != 3869]
img_list = mini_df.PATNO.tolist()
mini_df = mini_df.drop('PATNO', axis=1)

cohort_split = pd.get_dummies(mini_df['COHORT']) #Break cohort column into 2 columns (HC and PD)
cohort_split[0.0] = cohort_split[0.0].astype(int) #Boolean to Int (1/0)
cohort_split.rename(columns = {0.0:'PD'}, inplace = True) #rename 0 to PD
cohort_split[1.0] = cohort_split[1.0].astype(int)
cohort_split.rename(columns = {1.0:'HC'}, inplace = True)

mini_df = mini_df.drop('COHORT', axis=1)
frames = [mini_df, cohort_split]
final_df = pd.concat(frames, axis=1)
final_df.rename(columns = {'AGE_AT_VISIT':'AGE'}, inplace = True)
final_df['SEX'] = final_df['SEX'].astype(int) #combine the two dfs

display(final_df)
final_df.to_csv('/home/jacob/Desktop/analysis/baseline_DM.csv', index=False) #Save the df as a CSV
csv_to_mat('/home/jacob/Desktop/analysis/baseline_DM.csv', '/home/jacob/Desktop/analysis/baseline_DM.mat') # Convert the CSV to mat

,AGE,SEX,PD,HC
1299,70.2,0,0,1
1619,69.3,1,1,0
1655,57.2,1,1,0
1736,49.1,0,1,0
1753,60.0,0,1,0
2253,51.4,1,1,0
3565,52.4,1,0,1
3771,74.7,1,0,1
4023,78.8,1,0,1
4066,56.0,0,0,1


Generated /home/jacob/Desktop/analysis/baseline_DM.mat


In [8]:
#t-contrast
tcontrast_long = "/NumWaves 4\n/NumPoints 2\n/Matrix\n0 0 1 -1\n0 0 -1 1"
with open("/home/jacob/Desktop/analysis/baseline_t.con", "w") as text_file:
    text_file.write(tcontrast_long)

In [12]:
dir_list = []
for i in img_list:
    #temp_path = "./tessa-etal-2014/ANTs-derivatives/WR_to_GT/sub-" + str(i) + "/ses-BL/anat/WR2GT_1InverseWarp_jacobian_WR.nii.gz"
    temp_path = "/home/jacob/Desktop/analysis/jacobian_BL_to_GT/sub-" + str(i) + "/ses-BL/anat/BL2GT_1Warp_jacobian.nii.gz"
    dir_list.append(temp_path)
#make_4d(dir_list, './tessa-etal-2014/ANTs-derivatives/WR2GT_4D_corr.nii.gz')
make_4d(dir_list, '/home/jacob/Desktop/analysis/BL2GTJac_4D_ALL.nii.gz')

/home/jacob/Desktop/analysis/jacobian_BL_to_GT/sub-3106/ses-BL/anat/BL2GT_1Warp_jacobian.nii.gz
/home/jacob/Desktop/analysis/jacobian_BL_to_GT/sub-3123/ses-BL/anat/BL2GT_1Warp_jacobian.nii.gz
/home/jacob/Desktop/analysis/jacobian_BL_to_GT/sub-3124/ses-BL/anat/BL2GT_1Warp_jacobian.nii.gz
/home/jacob/Desktop/analysis/jacobian_BL_to_GT/sub-3127/ses-BL/anat/BL2GT_1Warp_jacobian.nii.gz
/home/jacob/Desktop/analysis/jacobian_BL_to_GT/sub-3128/ses-BL/anat/BL2GT_1Warp_jacobian.nii.gz
/home/jacob/Desktop/analysis/jacobian_BL_to_GT/sub-3179/ses-BL/anat/BL2GT_1Warp_jacobian.nii.gz
/home/jacob/Desktop/analysis/jacobian_BL_to_GT/sub-3301/ses-BL/anat/BL2GT_1Warp_jacobian.nii.gz
/home/jacob/Desktop/analysis/jacobian_BL_to_GT/sub-3316/ses-BL/anat/BL2GT_1Warp_jacobian.nii.gz
/home/jacob/Desktop/analysis/jacobian_BL_to_GT/sub-3350/ses-BL/anat/BL2GT_1Warp_jacobian.nii.gz
/home/jacob/Desktop/analysis/jacobian_BL_to_GT/sub-3353/ses-BL/anat/BL2GT_1Warp_jacobian.nii.gz
/home/jacob/Desktop/analysis/jacobian_BL

In [16]:
os.system("singularity run ../singufsl_img.simg randomise -i /home/jacob/Desktop/analysis/BL2GTJac_4D_ALL.nii.gz -o /home/jacob/Desktop/analysis/results1000/baseline_PDvsHC -d /home/jacob/Desktop/analysis/baseline_DM.mat -t /home/jacob/Desktop/analysis/baseline_t.con -n 1000 -m /home/jacob/Desktop/tessa-etal-2014_2/tessa-etal-2014/ANTs-derivatives/GT/GT_template0.nii.gz -T")

randomise options: -i /home/jacob/Desktop/analysis/BL2GTJac_4D_ALL.nii.gz -o /home/jacob/Desktop/analysis/results1000/baseline_PDvsHC -d /home/jacob/Desktop/analysis/baseline_DM.mat -t /home/jacob/Desktop/analysis/baseline_t.con -n 1000 -m /home/jacob/Desktop/tessa-etal-2014_2/tessa-etal-2014/ANTs-derivatives/GT/GT_template0.nii.gz -T 
Loading Data: 
Data loaded
1.28758e+10 permutations required for exhaustive test of t-test 1
Doing 1000 random permutations
Starting permutation 1 (Unpermuted data)
Starting permutation 2
Starting permutation 3
Starting permutation 4
Starting permutation 5
Starting permutation 6
Starting permutation 7
Starting permutation 8
Starting permutation 9
Starting permutation 10
Starting permutation 11
Starting permutation 12
Starting permutation 13
Starting permutation 14
Starting permutation 15
Starting permutation 16
Starting permutation 17
Starting permutation 18
Starting permutation 19
Starting permutation 20
Starting permutation 21
Starting permutation 22
S

Starting permutation 314
Starting permutation 315
Starting permutation 316
Starting permutation 317
Starting permutation 318
Starting permutation 319
Starting permutation 320
Starting permutation 321
Starting permutation 322
Starting permutation 323
Starting permutation 324
Starting permutation 325
Starting permutation 326
Starting permutation 327
Starting permutation 328
Starting permutation 329
Starting permutation 330
Starting permutation 331
Starting permutation 332
Starting permutation 333
Starting permutation 334
Starting permutation 335
Starting permutation 336
Starting permutation 337
Starting permutation 338
Starting permutation 339
Starting permutation 340
Starting permutation 341
Starting permutation 342
Starting permutation 343
Starting permutation 344
Starting permutation 345
Starting permutation 346
Starting permutation 347
Starting permutation 348
Starting permutation 349
Starting permutation 350
Starting permutation 351
Starting permutation 352
Starting permutation 353


Starting permutation 642
Starting permutation 643
Starting permutation 644
Starting permutation 645
Starting permutation 646
Starting permutation 647
Starting permutation 648
Starting permutation 649
Starting permutation 650
Starting permutation 651
Starting permutation 652
Starting permutation 653
Starting permutation 654
Starting permutation 655
Starting permutation 656
Starting permutation 657
Starting permutation 658
Starting permutation 659
Starting permutation 660
Starting permutation 661
Starting permutation 662
Starting permutation 663
Starting permutation 664
Starting permutation 665
Starting permutation 666
Starting permutation 667
Starting permutation 668
Starting permutation 669
Starting permutation 670
Starting permutation 671
Starting permutation 672
Starting permutation 673
Starting permutation 674
Starting permutation 675
Starting permutation 676
Starting permutation 677
Starting permutation 678
Starting permutation 679
Starting permutation 680
Starting permutation 681


Starting permutation 970
Starting permutation 971
Starting permutation 972
Starting permutation 973
Starting permutation 974
Starting permutation 975
Starting permutation 976
Starting permutation 977
Starting permutation 978
Starting permutation 979
Starting permutation 980
Starting permutation 981
Starting permutation 982
Starting permutation 983
Starting permutation 984
Starting permutation 985
Starting permutation 986
Starting permutation 987
Starting permutation 988
Starting permutation 989
Starting permutation 990
Starting permutation 991
Starting permutation 992
Starting permutation 993
Starting permutation 994
Starting permutation 995
Starting permutation 996
Starting permutation 997
Starting permutation 998
Starting permutation 999
Starting permutation 1000
Critical Value for: /home/jacob/Desktop/analysis/results1000/baseline_PDvsHC_tfce_tstat1 is: 30452.9
1.28758e+10 permutations required for exhaustive test of t-test 2
Doing 1000 random permutations
Starting permutation 1 (Un

Starting permutation 294
Starting permutation 295
Starting permutation 296
Starting permutation 297
Starting permutation 298
Starting permutation 299
Starting permutation 300
Starting permutation 301
Starting permutation 302
Starting permutation 303
Starting permutation 304
Starting permutation 305
Starting permutation 306
Starting permutation 307
Starting permutation 308
Starting permutation 309
Starting permutation 310
Starting permutation 311
Starting permutation 312
Starting permutation 313
Starting permutation 314
Starting permutation 315
Starting permutation 316
Starting permutation 317
Starting permutation 318
Starting permutation 319
Starting permutation 320
Starting permutation 321
Starting permutation 322
Starting permutation 323
Starting permutation 324
Starting permutation 325
Starting permutation 326
Starting permutation 327
Starting permutation 328
Starting permutation 329
Starting permutation 330
Starting permutation 331
Starting permutation 332
Starting permutation 333


Starting permutation 622
Starting permutation 623
Starting permutation 624
Starting permutation 625
Starting permutation 626
Starting permutation 627
Starting permutation 628
Starting permutation 629
Starting permutation 630
Starting permutation 631
Starting permutation 632
Starting permutation 633
Starting permutation 634
Starting permutation 635
Starting permutation 636
Starting permutation 637
Starting permutation 638
Starting permutation 639
Starting permutation 640
Starting permutation 641
Starting permutation 642
Starting permutation 643
Starting permutation 644
Starting permutation 645
Starting permutation 646
Starting permutation 647
Starting permutation 648
Starting permutation 649
Starting permutation 650
Starting permutation 651
Starting permutation 652
Starting permutation 653
Starting permutation 654
Starting permutation 655
Starting permutation 656
Starting permutation 657
Starting permutation 658
Starting permutation 659
Starting permutation 660
Starting permutation 661


Starting permutation 950
Starting permutation 951
Starting permutation 952
Starting permutation 953
Starting permutation 954
Starting permutation 955
Starting permutation 956
Starting permutation 957
Starting permutation 958
Starting permutation 959
Starting permutation 960
Starting permutation 961
Starting permutation 962
Starting permutation 963
Starting permutation 964
Starting permutation 965
Starting permutation 966
Starting permutation 967
Starting permutation 968
Starting permutation 969
Starting permutation 970
Starting permutation 971
Starting permutation 972
Starting permutation 973
Starting permutation 974
Starting permutation 975
Starting permutation 976
Starting permutation 977
Starting permutation 978
Starting permutation 979
Starting permutation 980
Starting permutation 981
Starting permutation 982
Starting permutation 983
Starting permutation 984
Starting permutation 985
Starting permutation 986
Starting permutation 987
Starting permutation 988
Starting permutation 989


0